## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-05-30-11 +0000,bbc,Ethiopian troops 'executed' aid workers in Tig...,https://www.bbc.com/news/articles/c8j1ve1w878o
1,2025-07-15-05-10-23 +0000,nypost,GOP congressman facing eviction blames broken ...,https://nypost.com/2025/07/15/us-news/gop-cong...
2,2025-07-15-05-07-31 +0000,bbc,Faroes-style tunnels could 'transform' fortune...,https://www.bbc.com/news/articles/cvg7jw27w1do
3,2025-07-15-05-07-07 +0000,bbc,"I'm 'disappointed but not done' with Putin, Tr...",https://www.bbc.com/news/articles/c1e02q12z32o
4,2025-07-15-04-30-50 +0000,bbc,China growth beats expectations as Trump tarif...,https://www.bbc.com/news/articles/c20r461g61xo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,14
92,russia,5
58,tariff,5
198,up,5
49,first,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
6,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...,49
55,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...,44
15,2025-07-15-00-53-28 +0000,nypost,NATO chief warns Russia’s trading partners aft...,https://nypost.com/2025/07/14/us-news/nato-chi...,38
36,2025-07-14-19-33-55 +0000,nypost,First lady Melania Trump helped convince presi...,https://nypost.com/2025/07/14/us-news/first-la...,32
11,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,31


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
6,49,2025-07-15-03-45-00 +0000,wsj,China’s economy showed resilience in a turbule...,https://www.wsj.com/world/china/chinas-economy...
55,30,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...
13,19,2025-07-15-01-25-19 +0000,nypost,Democratic city officers flocking to cop-frien...,https://nypost.com/2025/07/14/us-news/blue-cit...
15,19,2025-07-15-00-53-28 +0000,nypost,NATO chief warns Russia’s trading partners aft...,https://nypost.com/2025/07/14/us-news/nato-chi...
37,18,2025-07-14-19-28-42 +0000,nypost,Andrew Cuomo confirms independent run for NYC ...,https://nypost.com/2025/07/14/us-news/andrew-c...
64,18,2025-07-14-09-30-00 +0000,wsj,Food banks across the country were already str...,https://www.wsj.com/economy/food-banks-are-run...
53,17,2025-07-14-14-55-07 +0000,bbc,"Boeing fuel switches safe, regulator says afte...",https://www.bbc.com/news/articles/ce9xpgnx3vdo
69,17,2025-07-14-06-08-00 +0000,wsj,"China’s exports grew at a faster clip in June,...",https://www.wsj.com/economy/trade/chinas-expor...
47,15,2025-07-14-16-08-41 +0000,bbc,Dozens killed in clashes between Druze and Bed...,https://www.bbc.com/news/articles/cgmwp70pwmyo
30,13,2025-07-14-20-57-24 +0000,nypost,Sen. Rand Paul renews push for criminal probe ...,https://nypost.com/2025/07/14/us-news/sen-rand...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
